# 🗨️ Dialogue Summarization - Inference Demo

> 학습된 모델로 test 셋 예측 및 제출 파일 생성

이 노트북은 학습된 모델을 사용하여 test 데이터에 대한 요약을 생성합니다.

## 1. 환경 설정

In [ ]:
import sys
import os
from datetime import datetime

# 프로젝트 루트를 Python path에 추가
project_root = '/Competition/NLP/dialogue-summarization'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

## 2. 모듈 Import

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader

# 우리가 만든 모듈들
from src.data.preprocessor import Preprocess
from src.data.dataset import DatasetForInference
from src.models.model_loader import load_tokenizer_and_model
from src.evaluation.metrics import clean_text

print("✅ All modules imported successfully!")

## 3. 설정

In [ ]:
# 경로 설정
DATA_PATH = "/Competition/NLP/naturallanguageprocessingcompetition-nlp-1/data"
CHECKPOINT_PATH = "/Competition/NLP/dialogue-summarization/checkpoints/baseline_run/checkpoint-1750"  # Best model
OUTPUT_DIR = "/Competition/NLP/dialogue-summarization/submissions"

# 생성 파라미터
MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 100
NUM_BEAMS = 4
BATCH_SIZE = 32

# Special tokens
SPECIAL_TOKENS = ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']

print("📋 Configuration:")
print(f"  Data path: {DATA_PATH}")
print(f"  Checkpoint: {CHECKPOINT_PATH}")
print(f"  Output dir: {OUTPUT_DIR}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Num beams: {NUM_BEAMS}")

## 4. 디바이스 확인

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Device: {device}")

if device == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 5. 모델 & 토크나이저 로드

In [ ]:
print("📥 Loading model and tokenizer...")

model, tokenizer = load_tokenizer_and_model(
    model_name="digit82/kobart-summarization",  # Base model name for config
    special_tokens=SPECIAL_TOKENS,
    device=device,
    checkpoint_path=CHECKPOINT_PATH
)

model.eval()  # Evaluation mode

print(f"✅ Model loaded from: {CHECKPOINT_PATH}")
print(f"   Vocab size: {len(tokenizer)}")
print(f"   Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

## 6. Test 데이터 로드

In [ ]:
# Preprocessor 초기화
preprocessor = Preprocess(
    bos_token=tokenizer.bos_token,
    eos_token=tokenizer.eos_token
)

# Test 데이터 로드
test_data = preprocessor.make_set_as_df(f"{DATA_PATH}/test.csv", is_train=False)

print(f"📊 Test data loaded: {len(test_data)} samples")
print(f"\n🔍 Sample:")
print(f"   ID: {test_data.iloc[0]['fname']}")
print(f"   Dialogue: {test_data.iloc[0]['dialogue'][:100]}...")

## 7. 데이터 전처리

In [ ]:
# 전처리
encoder_input_test, _ = preprocessor.make_input(test_data, is_test=True)

# 토크나이징
tokenized_encoder_inputs = tokenizer(
    encoder_input_test,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=MAX_INPUT_LENGTH,
    return_token_type_ids=False
)

print(f"✅ Tokenization completed")
print(f"   Input shape: {tokenized_encoder_inputs['input_ids'].shape}")

## 8. Dataset & DataLoader 생성

In [ ]:
test_dataset = DatasetForInference(
    tokenized_encoder_inputs,
    test_data['fname'].tolist(),  # test_id로 fname 리스트 전달
    len(test_data)
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f"✅ DataLoader created")
print(f"   Total batches: {len(test_dataloader)}")

## 9. 추론 실행

In [ ]:
print("🚀 Starting inference...")

summaries = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Generating summaries"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # Generate
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=MAX_OUTPUT_LENGTH,
            num_beams=NUM_BEAMS,
            early_stopping=True,
            no_repeat_ngram_size=2
        )
        
        # Decode
        decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)
        summaries.extend(decoded)

print(f"\n✅ Inference completed!")
print(f"   Generated {len(summaries)} summaries")

## 10. 후처리

In [ ]:
# 토큰 정리
remove_tokens = ['<usr>', tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token]
cleaned_summaries = clean_text(summaries, remove_tokens)  # 리스트 전체를 한 번에 전달

print("✅ Post-processing completed")
print(f"\n🔍 Sample outputs:")
for i in range(min(3, len(cleaned_summaries))):
    print(f"\n[Sample {i+1}]")
    print(f"Original: {summaries[i][:80]}...")
    print(f"Cleaned:  {cleaned_summaries[i][:80]}...")

## 11. 제출 파일 생성

In [ ]:
# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# DataFrame 생성
submission_df = pd.DataFrame({
    'fname': test_data['fname'],
    'summary': cleaned_summaries
})

# 파일명 생성 (타임스탬프 포함)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"submission_{timestamp}.csv"
output_path = os.path.join(OUTPUT_DIR, output_filename)

# CSV 저장
submission_df.to_csv(output_path, index=False)

print(f"✅ Submission file saved!")
print(f"   Path: {output_path}")
print(f"   Rows: {len(submission_df)}")

## 12. 결과 확인

In [ ]:
# 제출 파일 미리보기
print("📄 Submission file preview:")
print(submission_df.head(10))

# 통계
print(f"\n📊 Statistics:")
print(f"   Total samples: {len(submission_df)}")
print(f"   Average summary length: {submission_df['summary'].str.len().mean():.1f} chars")
print(f"   Min summary length: {submission_df['summary'].str.len().min()} chars")
print(f"   Max summary length: {submission_df['summary'].str.len().max()} chars")

## 13. 샘플 비교

In [ ]:
# 몇 개 샘플 자세히 확인
import random
random.seed(42)

sample_indices = random.sample(range(len(test_data)), min(3, len(test_data)))

for idx in sample_indices:
    print(f"\n{'='*80}")
    print(f"Sample {idx + 1}")
    print(f"{'='*80}")
    print(f"\n[Dialogue]")
    print(test_data.iloc[idx]['dialogue'][:300] + "...")
    print(f"\n[Generated Summary]")
    print(submission_df.iloc[idx]['summary'])

## 완료!

### 다음 단계:
1. **제출 파일 확인**: `submissions/` 폴더의 CSV 파일 확인
2. **경진대회 제출**: 플랫폼에 CSV 파일 업로드
3. **결과 확인**: 리더보드에서 점수 확인